In [4]:
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv())
# print(os.getenv("GOOGLE_API_KEY"))

True

In [2]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


# Your first AI Agent with ADK

---

## 🤖 Section 2: Your first AI Agent with ADK

### 🤔 2.1 What is an AI Agent?

You've probably used an LLM like Gemini before, where you give it a prompt and it gives you a text response.

`Prompt -> LLM -> Text`

An AI Agent takes this one step further. An agent can think, take actions, and observe the results of those actions to give you a better answer.

`Prompt -> Agent -> Thought -> Action -> Observation -> Final Answer`

In this notebook, we'll build an agent that can take the action of searching Google. Let's see the difference!

In [13]:
## Configure Retry Options

# When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. 
# Retry options automatically handle these failures by retrying the request with exponential backoff.

retry_config = types.HttpRetryOptions(
    attempts=3,
    initial_delay=0.5,
    exp_base=7,
    )

In [ ]:
root_agent = Agent(
    name="helpful_assistant",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config
    ),
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent defined.")

✅ Root Agent defined.


### a. Create an InMemoryRunner and tell it to use our root_agent:

In [38]:
runner = InMemoryRunner(agent=root_agent)


print("✅ Runner created.")

✅ Runner created.


## b. Now you can call the .run_debug() method to send our prompt and get an answer.

In [39]:
response = await runner.run_debug(
    "What is Agent Development Kit from Google? What languages is the SDK available in?"
)


 ### Created new session: debug_session_id

User > What is Agent Development Kit from Google? What languages is the SDK available in?
helpful_assistant > The Agent Development Kit (ADK) from Google is an open-source, flexible, and modular framework designed for developing and deploying AI agents. Introduced at Google Cloud NEXT 2025, it aims to streamline the creation, deployment, and orchestration of agentic architectures, ranging from simple tasks to complex multi-agent systems. ADK applies software development principles to AI agent creation, offering developers precise control over agent behavior and orchestration. While optimized for Gemini and the Google ecosystem, ADK is model-agnostic, deployment-agnostic, and compatible with other frameworks. It also provides higher-level abstractions for agent development with built-in integrations for various models and tools.

The SDK for Google's Agent Development Kit is currently available in the following languages:
*   Python
*   TypeS

In [40]:
response = await runner.run_debug("What's the weather in Gondia Maharashtra?")


 ### Continue session: debug_session_id

User > What's the weather in Gondia Maharashtra?
helpful_assistant > The weather in Gondia, Maharashtra, is currently clear with a temperature of 14.4°C (58°F) and humidity around 42%.

For today, January 15, 2026, Gondia is expected to be sunny, with daytime temperatures reaching around 28.3°C (83°F) and dropping to 14.4°C (58°F) at night. There is no precipitation expected.

The forecast for tomorrow, January 16, 2026, also predicts sunny and dry conditions, with a high of approximately 28.4°C (83°F) and a low of 12.7°C (55°F). The humidity will be around 31%.


In [41]:
response = await runner.run_debug("Where is gondia located?")


 ### Continue session: debug_session_id

User > Where is gondia located?
helpful_assistant > Gondia is a city and the headquarters of the eponymous Gondia District, located in the state of Maharashtra, India. It is situated in the northeastern part of Maharashtra and shares its borders with the states of Madhya Pradesh to the north and Chhattisgarh to the east.

The city is part of the Nagpur Division and is often considered the gateway to Maharashtra from central and eastern India. Its geographical coordinates are approximately 21.459801° N latitude and 80.195000° E longitude.


# Multi-Agent Systems & Workflow Patterns

🤔 Section 2: Why Multi-Agent Systems? + Your First Multi-Agent¶


The Problem: The "Do-It-All" Agent

Single agents can do a lot. But what happens when the task gets complex? A single "monolithic" agent that tries to do research, writing, editing, and fact-checking all at once becomes a problem. Its instruction prompt gets long and confusing. It's hard to debug (which part failed?), difficult to maintain, and often produces unreliable results.

The Solution: A Team of Specialists

Instead of one "do-it-all" agent, we can build a multi-agent system. This is a team of simple, specialized agents that collaborate, just like a real-world team. Each agent has one clear job (e.g., one agent only does research, another only writes). This makes them easier to build, easier to test, and much more powerful and reliable when working together

In [2]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [3]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

Example: Research & Summarization System¶
Let's build a system with two specialized agents:

Research Agent - Searches for information using Google Search
Summarizer Agent - Creates concise summaries from research findings

In [5]:
# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

✅ research_agent created.


In [7]:
# Summarizer Agent: Its job is to summarize the text it receives.
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")

✅ summarizer_agent created.


In [8]:
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config
    ),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("✅ root_agent created.")

✅ root_agent created.


In [10]:
# Let's run the agent ask about a particular topic   
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "What are the latest car launches in INDIA?"
)


 ### Created new session: debug_session_id

User > What are the latest car launches in INDIA?
ResearchCoordinator > Here's a concise summary of the latest car launches in India:

*   India experienced numerous new car launches and updated variants in late 2025 and early 2026, featuring a mix of facelifts, new models, and electric vehicles.
*   January 2026 saw significant introductions, including the Tata Punch facelift (with new turbo petrol and CNG-AMT options), the new Mahindra XUV 7XO, an updated Kia Seltos, and the electric Mahindra XUV 3XO EV.
*   Other recent launches in late 2025 included the Tata Sierra, luxury electric variants like the Mercedes-Benz EQS SUV, and new MG Hector models.
*   The market shows a strong emphasis on SUVs and electric vehicles, a trend projected to continue with many anticipated launches throughout 2026, such as the Renault Duster and Tata Sierra EV.


 Section 3: Sequential Workflows - The Assembly Line

The Problem: Unpredictable Order

The previous multi-agent system worked, but it relied on a detailed instruction prompt to force the LLM to run steps in order. This can be unreliable. A complex LLM might decide to skip a step, run them in the wrong order, or get "stuck," making the process unpredictable.

The Solution: A Fixed Pipeline

When you need tasks to happen in a guaranteed, specific order, you can use a SequentialAgent. This agent acts like an assembly line, running each sub-agent in the exact order you list them. The output of one agent automatically becomes the input for the next, creating a predictable and reliable workflow.

# Example: Blog Post Creation with Sequential Agents¶
Let's build a system with three specialized agents:

Outline Agent - Creates a blog outline for a given topic
Writer Agent - Writes a blog post
Editor Agent - Edits a blog post draft for clarity and structure

In [12]:
# Outline Agent: Creates the initial blog post outline.
outline_agent = Agent(
    name="OutlineAgent",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config
    ),
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline",  # The result of this agent will be stored in the session state with this key.
)

print("✅ outline_agent created.")

✅ outline_agent created.


In [13]:
# Writer Agent: Writes the full blog post based on the outline from the previous agent.
writer_agent = Agent(
    name="WriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The `{blog_outline}` placeholder automatically injects the state value from the previous agent's output.
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    output_key="blog_draft",  # The result of this agent will be stored with this key.
)

print("✅ writer_agent created.")

✅ writer_agent created.


In [15]:
# Editor Agent: Edits and polishes the draft from the writer agent.
editor_agent = Agent(
    name="EditorAgent",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config
    ),
    # This agent receives the `{blog_draft}` from the writer agent's output.
    instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog",  # This is the final output of the entire pipeline.
)

print("✅ editor_agent created.")

✅ editor_agent created.


In [16]:
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


In [17]:
#Let's run the agent and give it a topic to write a blog post about:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a blog post about the benefits of multi-agent systems for software developers"
)


 ### Created new session: debug_session_id

User > Write a blog post about the benefits of multi-agent systems for software developers
OutlineAgent > Here's a blog outline about the benefits of multi-agent systems for software developers:

---

## Blog Outline: Multi-Agent Systems

**1. Catchy Headline:**
Level Up Your Dev Game: Why Multi-Agent Systems Are Your Next Software Superpower

**2. Introduction Hook:**
Are you tired of grappling with monolithic applications that are a nightmare to scale, maintain, and debug? Do you dream of building intelligent, resilient, and truly adaptive software? Modern software development is a battlefield of complexity, but a powerful paradigm shift is emerging to empower developers like never before: Multi-Agent Systems (MAS). Let's explore how MAS can revolutionize your development process and the applications you build.

**3. Main Sections:**

*   **Section 1: Unlocking Scalability and Modularity with Ease**
    *   **Breaking Down Complexity:** Mu

Section 4: Parallel Workflows - Independent Researchers

The Problem: The Bottleneck

The previous sequential agent is great, but it's an assembly line. Each step must wait for the previous one to finish. What if you have several tasks that are not dependent on each other? For example, researching three different topics. Running them in sequence would be slow and inefficient, creating a bottleneck where each task waits unnecessarily.

The Solution: Concurrent Execution

When you have independent tasks, you can run them all at the same time using a ParallelAgent. This agent executes all of its sub-agents concurrently, dramatically speeding up the workflow. Once all parallel tasks are complete, you can then pass their combined results to a final 'aggregator' step.

Use Parallel when: Tasks are independent, speed matters, and you can execute concurrently.

# Example: Parallel Multi-Topic Research
Let's build a system with four agents:

###Tech Researcher - Researches AI/ML news and trends

###Health Researcher - Researches recent medical news and trends

###Finance Researcher - Researches finance and fintech news and trends

###Aggregator Agent - Combines all research findings into a single summary

In [19]:
# Tech Researcher: Focuses on AI and ML trends.
tech_researcher = Agent(
    name="TechResearcher",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config
    ),
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research",  # The result of this agent will be stored in the session state with this key.
)

print("✅ tech_researcher created.")

✅ tech_researcher created.


In [20]:
# Health Researcher: Focuses on medical breakthroughs.
health_researcher = Agent(
    name="HealthResearcher",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config
    ),
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research",  # The result will be stored with this key.
)

print("✅ health_researcher created.")

✅ health_researcher created.


In [21]:
# Finance Researcher: Focuses on fintech trends.
finance_researcher = Agent(
    name="FinanceResearcher",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config
    ),
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research",  # The result will be stored with this key.
)

print("✅ finance_researcher created.")

✅ finance_researcher created.


In [22]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary",  # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


Then we bring the agents together under a parallel agent, which is itself nested inside of a sequential agent.

This design ensures that the research agents run first in parallel, then once all of their research is complete, the aggregator agent brings together all of the research findings into a single report:

In [23]:
# The ParallelAgent runs all its sub-agents simultaneously.
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

✅ Parallel and Sequential Agents created.


In [24]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Run the daily executive briefing on Tech, Health, and Finance"
)


 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health, and Finance
TechResearcher > The AI/ML landscape is rapidly evolving with three key developments. Firstly, **Generative AI** is maturing from pilot projects to deeply integrated, agentic systems across enterprises, exemplified by OpenAI's GPT models, Google DeepMind's Gemini, and Microsoft's Copilot, driving significant productivity and automation. Secondly, AI is revolutionizing **drug discovery and scientific research**, accelerating target identification and compound generation, with companies like Novartis and Recursion Pharmaceuticals leading the charge, promising faster, more cost-effective medical breakthroughs. Lastly, **AI ethics, governance, and regulation** are becoming paramount, with frameworks like the EU AI Act establishing transparency and accountability, influencing major cloud providers (AWS, Azure, GCP) and specialized firms like Pacific AI to ensure responsible AI d

# Section 5: Loop Workflows - The Refinement Cycle

The Problem: One-Shot Quality

All the workflows we've seen so far run from start to finish. The SequentialAgent and ParallelAgent produce their final output and then stop. This 'one-shot' approach isn't good for tasks that require refinement and quality control. What if the first draft of our story is bad? We have no way to review it and ask for a rewrite.

The Solution: Iterative Refinement

When a task needs to be improved through cycles of feedback and revision, you can use a LoopAgent. A LoopAgent runs a set of sub-agents repeatedly until a specific condition is met or a maximum number of iterations is reached. This creates a refinement cycle, allowing the agent system to improve its own work over and over.

Use Loop when: Iterative improvement is needed, quality refinement matters, or you need repeated cycles.

### 5.1 Example: Iterative Story Refinement

Let's build a system with two agents:

1. **Writer Agent** - Writes a draft of a short story
2. **Critic Agent** - Reviews and critiques the short story to suggest improvements

### 5.1 Example: Iterative Story Refinement

Let's build a system with two agents:

1. **Writer Agent** - Writes a draft of a short story
2. **Critic Agent** - Reviews and critiques the short story to suggest improvements

In [4]:
# This agent runs ONCE at the beginning to create the first draft.
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config
    ),
    instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story",  # Stores the first draft in the state.
)

print("✅ initial_writer_agent created.")

✅ initial_writer_agent created.


In [5]:
# This agent's only job is to provide feedback or the approval signal. It has no tools.
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config
    ),
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}
    
    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique",  # Stores the feedback in the state.
)

print("✅ critic_agent created.")

✅ critic_agent created.


Now, we need a way for the loop to actually stop based on the critic's feedback. The LoopAgent itself doesn't automatically know that "APPROVED" means "stop."

We need an agent to give it an explicit signal to terminate the loop.

We do this in two parts:

A simple Python function that the LoopAgent understands as an "exit" signal.
An agent that can call that function when the right condition is met.
First, you'll define the exit_loop function:

In [6]:
# This is the function that the RefinerAgent will call to exit the loop.
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}


print("✅ exit_loop function created.")

✅ exit_loop function created.


To let an agent call this Python function, we wrap it in a FunctionTool. Then, we create a RefinerAgent that has this tool.

👉 Notice its instructions: this agent is the "brain" of the loop. It reads the {critique} from the CriticAgent and decides whether to (1) call the exit_loop tool or (2) rewrite the story.

In [7]:
# This agent refines the story based on critique OR calls the exit_loop function.
refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config
    ),
    instruction="""You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    output_key="current_story",  # It overwrites the story with the new, refined version.
    tools=[
        FunctionTool(exit_loop)
    ],  # The tool is now correctly initialized with the function reference.
)

print("✅ refiner_agent created.")

✅ refiner_agent created.


Then we bring the agents together under a loop agent, which is itself nested inside of a sequential agent.

This design ensures that the system first produces an initial story draft, then the refinement loop runs up to the specified number of max_iterations:

In [8]:
# The LoopAgent contains the agents that will run repeatedly: Critic -> Refiner.
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2,  # Prevents infinite loops
)

# The root agent is a SequentialAgent that defines the overall workflow: Initial Write -> Refinement Loop.
root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


In [12]:
# Let's run the agent and give it a topic to write a short story about:

In [9]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a short story about a lighthouse keeper who discovers a mysterious, glowing map"
)


 ### Created new session: debug_session_id

User > Write a short story about a lighthouse keeper who discovers a mysterious, glowing map
InitialWriterAgent > Thomas, keeper of the isolated Black Rock Lighthouse, knew the rhythm of the waves and the solace of solitude. One blustery evening, as he polished the lantern room's brass, his gaze caught an unusual glimmer beneath a stack of old logbooks. He reached, pulling out a rolled parchment, brittle with age.

Unfurling it carefully, Thomas’s breath hitched. It was a map, hand-drawn and intricate, detailing uncharted islands and swirling oceanic currents. What truly captivated him was its luminescence. Faint, ethereal lines on the parchment pulsed with a soft, cerulean glow, highlighting cryptic symbols and a single, shimmering 'X' marked far into the unknown expanse. The ancient chart hummed with an otherworldly energy, beckoning him towards mysteries beyond the horizon. His quiet life had just found a new direction.
CriticAgent > This

c:\Users\mjkat\Documents\TO DO PROJECTS\A Kaggle  Project Agent\Agent 5day\myenv\Lib\site-packages\google\adk\flows\llm_flows\base_llm_flow.py:449: UserWarning: [EXPERIMENTAL] feature FeatureName.PROGRESSIVE_SSE_STREAMING is enabled.
  async for event in agen:


Summary - Choosing the Right Pattern


Decision Tree: Which Workflow Pattern?

### Quick Reference Table

| Pattern | When to Use | Example | Key Feature |
|---------|-------------|---------|-------------|
| **LLM-based (sub_agents)** | Dynamic orchestration needed | Research + Summarize | LLM decides what to call |
| **Sequential** | Order matters, linear pipeline | Outline -> Write -> Edit | Deterministic order |
| **Parallel** | Independent tasks, speed matters | Multi-topic research | Concurrent execution |
| **Loop** | Iterative improvement needed | Writer + Critic refinement | Repeated cycles |